# Scraping Transfermarkt

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Functions

In [18]:
def top_n_leagues(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []
    League = pageSoup.find_all("img", {"class": "continental-league-emblem"})

    if n > len(League):
        print("There is not that many leagues")
    else:
        for i in range(0,n):
            LeagueList.append(str(League[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return LeagueList

In [19]:
def top_n_leagues_links(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []

    for name in n:
        League = pageSoup.find_all("a", {"title": name})
        for i in range(0,len(League)):
            LeagueList.append("https://www.transfermarkt.co.uk" + str(League[i]).split('" title', 1)[0].split('href="', 1)[1])

    return LeagueList

In [20]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

In [21]:
# Prints link only
def team_links(links, team_names):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        for name in team_names:
            Team = pageSoup.find_all("a", {"title": name})
            x = 0
            for i in range(0,len(Team)):
                TeamList.append("https://www.transfermarkt.co.uk" + str(Team[i]).split('" title',1)[0].split('href="', 1)[1])
                x += 1
                if x > 0:
                    break

    return TeamList

In [22]:
def build_df(links):

    PlayersList = []
    AgeList=[]
    NationList=[]
    ValuesList = []
    PositionsList=[]
    ClubList = []

    for a in links:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        Age = pageSoup.find_all("td", {"class": "zentriert"})
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
        Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
        Nationality = pageSoup.find_all("td", {"class": "zentriert"})
        Club = pageSoup.find_all("meta", {"name": "keywords"})

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            #print(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1], Values[i].text)
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

        for i in range(0,len(Positions)):
            PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

        for i in range(1,(len(Players)*3),3):
            #print(len(Age), len(str(Age[i]).split("(",1)), len(str(Age[i]).split("(",1)[1].split(")",1)), i)
            AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

        for i in range(2,(len(Players)*3),3):
            NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])
        for i in range(0,len(Players)):
            ClubList.append(str(Club[0]).split('content="')[1].split(",")[0])

    #Initial uncleaned dataframe initiated
    df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList, "Values":ValuesList,})

    df.loc[(df.Values == '\xa0'),'Values']= None
        
    return df

# Premier League

In [23]:
leagues = top_n_leagues(10)
leagues = leagues[0:1]

league_links = top_n_leagues_links(leagues)

team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'
        
every_team_links = every_team_links[0:20]

df = build_df(every_team_links)

df = df.dropna()
df

,Players,Club,Position,Age,Nationality,Values
0,Ederson,Manchester City,Goalkeeper,29,Brazil,€40.00m
1,Stefan Ortega,Manchester City,Goalkeeper,30,Germany,€9.00m
2,Zack Steffen,Manchester City,Goalkeeper,28,United States,€3.50m
3,Scott Carson,Manchester City,Goalkeeper,37,England,€200k
4,Rúben Dias,Manchester City,Defender,26,Portugal,€80.00m
...,...,...,...,...,...,...
573,Reiss Nelson,Arsenal FC,Attack,23,England,€12.00m
574,Marquinhos,Arsenal FC,Attack,20,Brazil,€12.00m
575,Gabriel Jesus,Arsenal FC,Attack,26,Brazil,€75.00m
576,Folarin Balogun,Arsenal FC,Attack,22,United States,€30.00m


# La Liga

In [24]:
leagues = top_n_leagues(10)
leagues = leagues[1:2]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:20]

df2 = build_df(every_team_links)

df2 = df2.dropna()
df2

,Players,Club,Position,Age,Nationality,Values
0,Thibaut Courtois,Real Madrid,Goalkeeper,31,Belgium,€45.00m
1,Andriy Lunin,Real Madrid,Goalkeeper,24,Ukraine,€5.00m
2,Éder Militão,Real Madrid,Defender,25,Brazil,€70.00m
3,David Alaba,Real Madrid,Defender,31,Austria,€40.00m
4,Antonio Rüdiger,Real Madrid,Defender,30,Germany,€32.00m
...,...,...,...,...,...,...
515,Luis Rioja,Deportivo Alavés,Attack,29,Spain,€3.00m
516,Abde Rebbach,Deportivo Alavés,Attack,24,Algeria,€500k
517,Xeber Alkain,Deportivo Alavés,Attack,26,Spain,€1.60m
518,Miguel de la Fuente,Deportivo Alavés,Attack,23,Spain,€2.00m


# Serie A

In [25]:
leagues = top_n_leagues(10)
leagues = leagues[2:3]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:20]

df3 = build_df(every_team_links)

df3 = df3.dropna()
df3

,Players,Club,Position,Age,Nationality,Values
0,Alex Meret,SSC Napoli,Goalkeeper,26,Italy,€18.00m
1,Nikita Contini,SSC Napoli,Goalkeeper,27,Italy,€500k
2,Hubert Idasiak,SSC Napoli,Goalkeeper,21,Poland,€150k
3,Min-jae Kim,SSC Napoli,Defender,26,"Korea, South",€60.00m
4,Amir Rrahmani,SSC Napoli,Defender,29,Kosovo,€25.00m
...,...,...,...,...,...,...
625,Giorgi Kvernadze,Frosinone Calcio,Attack,20,Georgia,€500k
626,Soufiane Bidaoui,Frosinone Calcio,Attack,33,Morocco,€400k
627,Hamza Haoudi,Frosinone Calcio,Attack,22,Italy,€175k
628,Gennaro Borrelli,Frosinone Calcio,Attack,23,Italy,€500k


# Bundasliga

In [26]:
leagues = top_n_leagues(10)
leagues = leagues[3:4]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df4 = build_df(every_team_links)

df4 = df4.dropna()
df4

,Players,Club,Position,Age,Nationality,Values
0,Alexander Nübel,Bayern Munich,Goalkeeper,26,Germany,€8.00m
1,Manuel Neuer,Bayern Munich,Goalkeeper,37,Germany,€7.00m
2,Yann Sommer,Bayern Munich,Goalkeeper,34,Switzerland,€5.00m
3,Sven Ulreich,Bayern Munich,Goalkeeper,34,Germany,€700k
4,Johannes Schenk,Bayern Munich,Goalkeeper,20,Germany,€300k
...,...,...,...,...,...,...
551,Braydon Manu,SV Darmstadt 98,Attack,26,Ghana,€1.50m
552,Oscar Vilhelmsson,SV Darmstadt 98,Attack,19,Sweden,€1.50m
553,Fraser Hornby,SV Darmstadt 98,Attack,23,Scotland,€1.00m
554,Aaron Seydel,SV Darmstadt 98,Attack,27,Germany,€500k


# Ligue 1

In [27]:
leagues = top_n_leagues(10)
leagues = leagues[4:5]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df5 = build_df(every_team_links)

df5 = df5.dropna()
df5

,Players,Club,Position,Age,Nationality,Values
0,Gianluigi Donnarumma,Paris Saint-Germain,Goalkeeper,24,Italy,€45.00m
1,Keylor Navas,Paris Saint-Germain,Goalkeeper,36,Costa Rica,€4.00m
2,Sergio Rico,Paris Saint-Germain,Goalkeeper,29,Spain,€3.00m
3,Lucas Lavallée,Paris Saint-Germain,Goalkeeper,20,France,€300k
4,Alexandre Letellier,Paris Saint-Germain,Goalkeeper,32,France,€300k
...,...,...,...,...,...,...
506,Vagner,FC Metz,Attack,27,Cape Verde,€650k
507,Warren Tchimbembé,FC Metz,Attack,25,Congo,€500k
508,Georges Mikautadze,FC Metz,Attack,22,Georgia,€10.00m
509,Simon Elisor,FC Metz,Attack,23,France,€800k


# Liga Portugal

In [28]:
leagues = top_n_leagues(10)
leagues = leagues[5:6]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df6 = build_df(every_team_links)

df6 = df6.dropna()
df6

,Players,Club,Position,Age,Nationality,Values
0,Odysseas Vlachodimos,SL Benfica,Goalkeeper,29,Greece,€16.00m
1,Samuel Soares,SL Benfica,Goalkeeper,21,Portugal,€2.00m
2,André Gomes,SL Benfica,Goalkeeper,18,Portugal,€2.00m
3,António Silva,SL Benfica,Defender,19,Portugal,€45.00m
4,Morato,SL Benfica,Defender,22,Brazil,€13.00m
...,...,...,...,...,...,...
492,Cristian Ponde,SC Farense,Attack,28,Portugal,€600k
493,Belloumi,SC Farense,Attack,21,Algeria,€300k
494,Elves Baldé,SC Farense,Attack,23,Portugal,€275k
496,Rui Costa,SC Farense,Attack,27,Portugal,€550k


# Eredivisie

In [29]:
leagues = top_n_leagues(10)
leagues = leagues[6:7]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df7 = build_df(every_team_links)

df7 = df7.dropna()
df7

,Players,Club,Position,Age,Nationality,Values
0,Gerónimo Rulli,Ajax Amsterdam,Goalkeeper,31,Argentina,€10.00m
1,Jay Gorter,Ajax Amsterdam,Goalkeeper,23,Netherlands,€1.00m
2,Remko Pasveer,Ajax Amsterdam,Goalkeeper,39,Netherlands,€250k
3,Jurrien Timber,Ajax Amsterdam,Defender,22,Netherlands,€42.00m
4,Calvin Bassey,Ajax Amsterdam,Defender,23,Nigeria,€15.00m
...,...,...,...,...,...,...
448,Denilho Cleonise,RKC Waalwijk,Attack,21,Netherlands,€350k
449,Roy Kuijpers,RKC Waalwijk,Attack,23,Netherlands,€300k
450,Michiel Kramer,RKC Waalwijk,Attack,34,Netherlands,€400k
451,David Min,RKC Waalwijk,Attack,24,Netherlands,€225k


 # Süper Lig

In [30]:
leagues = top_n_leagues(10)
leagues = leagues[7:8]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df8 = build_df(every_team_links)

df8 = df8.dropna()
df8

,Players,Club,Position,Age,Nationality,Values
0,Altay Bayindir,Fenerbahce,Goalkeeper,25,Turkey,€11.00m
1,Irfan Can Egribayat,Fenerbahce,Goalkeeper,25,Turkey,€1.90m
2,Furkan Akyüz,Fenerbahce,Goalkeeper,17,Turkey,€50k
3,Attila Szalai,Fenerbahce,Defender,25,Hungary,€14.00m
4,Alexander Djiku,Fenerbahce,Defender,28,Ghana,€8.00m
...,...,...,...,...,...,...
581,Sefa Yilmaz,Caykur Rizespor,Attack,33,Turkey,€75k
582,Ugur Can Semizoglu,Caykur Rizespor,Attack,22,Turkey,€25k
583,John Mary,Caykur Rizespor,Attack,30,Cameroon,€1.00m
584,Anthony van den Hurk,Caykur Rizespor,Attack,30,Curacao,€400k


# Jupiler Pro League

In [31]:
leagues = top_n_leagues(10)
leagues = leagues[9:10]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = every_team_links[0:18]

df9 = build_df(every_team_links)

df9 = df9.dropna()
df9

,Players,Club,Position,Age,Nationality,Values
0,Simon Mignolet,Club Brugge KV,Goalkeeper,35,Belgium,€3.20m
1,Josef Bursik,Club Brugge KV,Goalkeeper,23,England,€1.50m
2,Nick Shinton,Club Brugge KV,Goalkeeper,22,Belgium,€300k
3,Abakar Sylla,Club Brugge KV,Defender,20,Cote d'Ivoire,€8.00m
4,Brandon Mechele,Club Brugge KV,Defender,30,Belgium,€4.50m
...,...,...,...,...,...,...
467,Gift Orban,KAA Gent,Attack,20,Nigeria,€20.00m
468,Hugo Cuypers,KAA Gent,Attack,26,Belgium,€11.00m
469,Tarik Tissoudali,KAA Gent,Attack,30,Morocco,€6.00m
470,Gianni Bruno,KAA Gent,Attack,31,Belgium,€1.00m


# Final DataFrame

In [32]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)

In [33]:
final_df.to_csv('transfermarkt_9_leagues_2023.csv', index = False)